# ¬@Task 1 
## Implementing a Custom Hash Table with Collision Handling

In [1]:
import time, random, string

def random_key(length=6):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

class ListNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class HashTableChaining:
    def __init__(self, size=1009):
        self.size = size
        self.table = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        head = self.table[idx]
        while head:
            if head.key == key:
                head.value = value
                return
            head = head.next
        new_node = ListNode(key, value)
        new_node.next = self.table[idx]
        self.table[idx] = new_node

    def get(self, key):
        idx = self._hash(key)
        head = self.table[idx]
        while head:
            if head.key == key:
                return head.value
            head = head.next
        return None

    def delete(self, key):
        idx = self._hash(key)
        prev = None
        curr = self.table[idx]
        while curr:
            if curr.key == key:
                if prev:
                    prev.next = curr.next
                else:
                    self.table[idx] = curr.next
                return
            prev, curr = curr, curr.next

    def display(self):
        for i, node in enumerate(self.table):
            chain = []
            while node:
                chain.append(f"{node.key}:{node.value}")
                node = node.next
            if chain:
                print(f"{i}: {' -> '.join(chain)}")

class HashTableLinearProbing:
    def __init__(self, size=2003):
        self.size = size
        self.table = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None or self.table[probe][0] == key:
                self.table[probe] = (key, value)
                return
        raise Exception("Hash table is full")

    def get(self, key):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None:
                return None
            if self.table[probe][0] == key:
                return self.table[probe][1]
        return None

    def delete(self, key):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None:
                return
            if self.table[probe][0] == key:
                self.table[probe] = ("__DELETED__", None)
                return

    def display(self):
        for i, item in enumerate(self.table):
            if item and item[0] != "__DELETED__":
                print(f"{i}: {item[0]}:{item[1]}")

def compare_hash_tables():
    print("Generating 1000 random key-value pairs...")
    data = [(random_key(), random.randint(1, 10000)) for _ in range(1000)]

    ht_chain = HashTableChaining()
    start = time.time()
    for k, v in data:
        ht_chain.insert(k, v)
    insert_time_chain = time.time() - start

    start = time.time()
    for k, _ in data:
        _ = ht_chain.get(k)
    search_time_chain = time.time() - start

    ht_probe = HashTableLinearProbing()
    start = time.time()
    for k, v in data:
        ht_probe.insert(k, v)
    insert_time_probe = time.time() - start

    start = time.time()
    for k, _ in data:
        _ = ht_probe.get(k)
    search_time_probe = time.time() - start

    print("\n--- Performance Comparison ---")
    print(f"Chaining:     Insert = {insert_time_chain:.6f}s | Search = {search_time_chain:.6f}s")
    print(f"Linear Probing: Insert = {insert_time_probe:.6f}s | Search = {search_time_probe:.6f}s")

compare_hash_tables()

Generating 1000 random key-value pairs...

--- Performance Comparison ---
Chaining:     Insert = 0.003990s | Search = 0.022495s
Linear Probing: Insert = 0.001985s | Search = 0.002996s


# ¬@Task 2
## Checking if Two Strings Are Anagrams Using Hashing

In [2]:
def are_anagrams(s1, s2):
    if len(s1) != len(s2):
        return False

    freq = {}

    for char in s1:
        freq[char] = freq.get(char, 0) + 1

    for char in s2:
        if char not in freq:
            return False
        freq[char] -= 1
        if freq[char] < 0:
            return False

    return True

def test_anagrams():
    test_cases = [
        ("listen", "silent", True),
        ("triangle", "integral", True),
        ("hello", "bello", False),
        ("aabbcc", "abcabc", True),
        ("aabbcc", "aabbc", False),
        ("", "", True),
        ("a", "a", True),
        ("a", "b", False),
        ("123!@#", "!@#321", True),
        ("anagram", "nagaram", True)
    ]

    print("Basic Test Cases:")
    for s1, s2, expected in test_cases:
        result = are_anagrams(s1, s2)
        print(f"{s1} & {s2} → {result} → {'Pass' if result == expected else 'Fail'}")

    print("\nTesting with large strings...")
    s1 = "a" * 100000 + "b" * 100000 + "c" * 100000
    s2 = "b" * 100000 + "c" * 100000 + "a" * 100000
    result = are_anagrams(s1, s2)
    print("Large strings →", "Pass" if result else "Fail")

test_anagrams()

Basic Test Cases:
listen & silent → True → Pass
triangle & integral → True → Pass
hello & bello → False → Pass
aabbcc & abcabc → True → Pass
aabbcc & aabbc → False → Pass
 &  → True → Pass
a & a → True → Pass
a & b → False → Pass
123!@# & !@#321 → True → Pass
anagram & nagaram → True → Pass

Testing with large strings...
Large strings → Pass


# ¬@Task 3
##  Implementing a Simple Caching Mechanism Using Hash Maps

In [3]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:
    def __init__(self, capacity=5):
        self.capacity = capacity
        self.cache = {}
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove(self, node):
        prev, nxt = node.prev, node.next
        prev.next = nxt
        nxt.prev = prev

    def _add_to_front(self, node):
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node

    def get(self, key):
        if key in self.cache:
            node = self.cache[key]
            self._remove(node)
            self._add_to_front(node)
            return node.value
        return -1

    def put(self, key, value):
        if key in self.cache:
            self._remove(self.cache[key])
        elif len(self.cache) == self.capacity:
            lru = self.tail.prev
            self._remove(lru)
            del self.cache[lru.key]

        new_node = Node(key, value)
        self._add_to_front(new_node)
        self.cache[key] = new_node

    def display(self):
        current = self.head.next
        print("Cache state (Most → Least recent):", end=" ")
        while current != self.tail:
            print(f"({current.key}:{current.value})", end=" → ")
            current = current.next
        print("NULL")

def test_lru_cache():
    lru = LRUCache(capacity=5)

    for i in range(1, 6):
        lru.put(i, i*100)
    lru.display()

    print("Access key 3:", lru.get(3))
    print("Access key 1:", lru.get(1))
    lru.display()

    for i in range(6, 11):
        lru.put(i, i*100)
        lru.display()

    print("Access key 2 (should be -1):", lru.get(2))
    print("Access key 10:", lru.get(10))
    lru.display()

test_lru_cache()

Cache state (Most → Least recent): (5:500) → (4:400) → (3:300) → (2:200) → (1:100) → NULL
Access key 3: 300
Access key 1: 100
Cache state (Most → Least recent): (1:100) → (3:300) → (5:500) → (4:400) → (2:200) → NULL
Cache state (Most → Least recent): (6:600) → (1:100) → (3:300) → (5:500) → (4:400) → NULL
Cache state (Most → Least recent): (7:700) → (6:600) → (1:100) → (3:300) → (5:500) → NULL
Cache state (Most → Least recent): (8:800) → (7:700) → (6:600) → (1:100) → (3:300) → NULL
Cache state (Most → Least recent): (9:900) → (8:800) → (7:700) → (6:600) → (1:100) → NULL
Cache state (Most → Least recent): (10:1000) → (9:900) → (8:800) → (7:700) → (6:600) → NULL
Access key 2 (should be -1): -1
Access key 10: 1000
Cache state (Most → Least recent): (10:1000) → (9:900) → (8:800) → (7:700) → (6:600) → NULL
